## Created nate_branch for phase 2 group project

## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [47]:
# Import the necessary libraries

import pandas as pd 
import matplotlib.pyplot as plt
import math
import scipy as sp
from scipy import stats
from scipy.stats import norm
import scipy.stats as stats
import numpy as np
import statsmodels.api as sm
import seaborn as sns
from collections import Counter
from math import gamma
from statsmodels.formula.api import ols

pd.set_option('display.max_columns', None)
plt.style.use('fivethirtyeight')
sns.set_style('whitegrid')
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [58]:
# read the data into a variable called data
data = pd.read_csv('data/kc_house_data.csv')

# make a copy of data to work with calle df
df = data.copy(deep=True)

In [59]:
df.shape

(21597, 21)

In [60]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')